In [1]:
### If Google Colab
# !git config --global user.email "sjs.shamsi@gmail.com"
# !git config --global user.name "SJShamsi"

# from google.colab import drive
# drive.mount('/content/drive')

# Thesis_Files = '/content/drive/MyDrive/Thesis_Files/'

In [2]:
import sys
sys.path.insert(0, '/home/shoaib/Thesis_Files/Thesis/dmdt_Analysis/')
# sys.path.insert(0, Thesis_Files + 'Thesis/dmdt_Analysis/')

from dmdt_functions import *

In [3]:
from pandas import read_parquet, DataFrame, concat
from numpy import linspace, load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')
plt.style.use('seaborn-v0_8-colorblind')
plt.rcParams['axes.grid'] = False

In [4]:
all_lightcurves_file = "/home/shoaib/Thesis_Files/Thesis/Preprocessing/all_lightcurves_10min_cadence.pkl"
# filtered_lightcurves_file = Thesis_Files + "/filtered_lightcurves.parquet"

### The following bins are simply linear bins
# nxbins, nybins = 51, 51
# dm_bins = linspace(-1, 1, nybins)
# dt_bins = linspace(0, 1160, nxbins)

### The following bins ensure each bins has equal dm and dt occurances for our entire sample
# dm_bins = load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/equivalent_bins_50x50_QSOs/QSO_dm_bins.npy')
# dt_bins = load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/equivalent_bins_50x50_QSOs/QSO_dt_bins.npy')

### The following bins are from the Mahabal repository
# dm_bins = load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/dm_-1_1_Mahabal_bins_22.npy')
# dt_bins = load(Thesis_Files + 'Thesis/dmdt_Analysis/Bins/dt_0_1160_Mahabal_bins_24.npy')
dm_bins = load('/home/shoaib/Thesis_Files/Thesis/dmdt_Analysis/Bins/dm_-1_1_Mahabal_bins_22.npy')
dt_bins = load('/home/shoaib/Thesis_Files/Thesis/dmdt_Analysis/Bins/dt_0_1160_Mahabal_bins_24.npy')

### the following are some custom experimental bins
# dm_bins = np.array([])
# dt_bins = np.array([])

# These files do'nt exist yet, they are save paths for later in the notebook
# dmdt_by_Name_Band_50x50_equivalent_bins = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_equivalent_bins.pkl'
# dmdt_by_Name_Band_linear_bins = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50_linear_bins.pkl'
# dmdt_file = Thesis_Files + 'Thesis/dmdt_Analysis/Histograms/dmdt_histograms_21x23_Mahabal_bins.pkl'
dmdt_file = '/home/shoaib/Thesis_Files/Thesis/dmdt_Analysis/Histograms/dmdt_histograms_1day_cadence_21x23_Mahabal_bins.pkl'

In [5]:
### By experimentation we know that these are good bins

all_lightcurves = pd.read_pickle(all_lightcurves_file)

# frac = 0.2
# unique_names = all_lightcurves['name'].unique()
# random_names = np.random.choice(unique_names, size=int(len(unique_names) * frac), replace=False)
# all_lightcurves = all_lightcurves[all_lightcurves['name'].isin(random_names)]

In [6]:
all_lightcurves.head()

,name,type,band,n_good_det,light_curve
0,1AXG J000605+2031,AX,g,212,"[[58268.484, 17.2914085, 0.0193178076], [58276..."
1,1AXG J000605+2031,AX,r,207,"[[58268.456, 17.2935829, 0.0198369212], [58270..."
2,1AXG J102337+1936,AX,g,205,"[[58202.204, 19.304962769298335, 0.03818369785..."
3,1AXG J102337+1936,AX,r,220,"[[58217.145, 19.046276484193328, 0.04050197847..."
4,1AXG J103934+5330,AX,g,233,"[[58202.215, 18.849639899999996, 0.04993807277..."


In [7]:
### Looping now
grouped = all_lightcurves.groupby(by='name')
histogram_dict_list = []
num_total_groups = len(grouped)
count = 0

for name, df in grouped:
    count += 1
    print(f'{count}\t/ {num_total_groups} objects in their bands done.')

    bands = df['band'].to_numpy()

    for band in bands:
        object_type = df['type'].iloc[0]
        light_curve = df.loc[df['band'] == band]['light_curve'].values[0]
        n_good_det = df.shape[0]
        mags, times = light_curve[:, 1], light_curve[:, 0]
        
        hist, _junk, _junk = get_dmdt_histogram(magnitudes=mags, dmagnitude_bins=dm_bins, times=times, dtime_bins=dt_bins)
        histogram_dict = {'name': name, 'type': object_type, 'band': band, 'n_good_det': n_good_det, 'histogram': hist}
        histogram_dict_list.append(histogram_dict)

1	/ 31619 objects in their bands done.
2	/ 31619 objects in their bands done.
3	/ 31619 objects in their bands done.
4	/ 31619 objects in their bands done.
5	/ 31619 objects in their bands done.
6	/ 31619 objects in their bands done.
7	/ 31619 objects in their bands done.
8	/ 31619 objects in their bands done.
9	/ 31619 objects in their bands done.
10	/ 31619 objects in their bands done.
11	/ 31619 objects in their bands done.
12	/ 31619 objects in their bands done.
13	/ 31619 objects in their bands done.
14	/ 31619 objects in their bands done.
15	/ 31619 objects in their bands done.
16	/ 31619 objects in their bands done.
17	/ 31619 objects in their bands done.
18	/ 31619 objects in their bands done.
19	/ 31619 objects in their bands done.
20	/ 31619 objects in their bands done.
21	/ 31619 objects in their bands done.
22	/ 31619 objects in their bands done.
23	/ 31619 objects in their bands done.
24	/ 31619 objects in their bands done.
25	/ 31619 objects in their bands done.
26	/ 3161

In [ ]:
hist_df = DataFrame(histogram_dict_list)
histogram_dict_list = []
hist_df.head()

In [10]:
hist_df.reset_index(drop=True).to_pickle(dmdt_file)

### Looking at what the histograms look like

In [ ]:
hist_df = pd.read_pickle(dmdt_file)

In [ ]:
one_LC = all_lightcurves[(all_lightcurves['name'] == '2MASS J01283395-2358359') & (all_lightcurves['band'] == 'g')]
one_LC = one_LC.sort_values(by='mjd')

mags = one_LC['mag'].to_numpy()
times = one_LC['mjd'].to_numpy()

dm, dt = return_dms_dts(mags, times)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))

axs[0].scatter(times, mags)
axs[0].set_title('Magnitude vs. Times for a random object in g band')
axs[0].set_xlabel('Times')
axs[0].set_ylabel('Mag')

axs[1].scatter(dt, dm)
axs[1].set_title('$dm$ vs. $dt$ for a random object in g band')
axs[1].set_xlabel('$dt$')
axs[1].set_ylabel('$dm$')

In [ ]:
plt.hexbin(dt, dm, cmap='jet')

In [ ]:
hist, _, _ = np.histogram2d(dt, dm, bins=[23, 24])
plt.imshow(hist.T, origin='lower', interpolation='bilinear')

In [ ]:
mock_mags = [2, 2.1, 1.8, 3, 3, 2.2, 1.6]
mock_times = [1, 23, 34, 35, 36, 37, 38.8]

dm, dt = return_dms_dts(mock_mags, mock_times)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))

axs[0].scatter(mock_times, mock_mags)
axs[0].set_title('Mock Magnitude vs. Times')
axs[0].set_xlabel('Times')
axs[0].set_ylabel('Mag')

axs[1].scatter(dt, dm)
axs[1].set_title('Mock $dm$ vs. $dt$')
axs[1].set_xlabel('$dt$')
axs[1].set_ylabel('$dm$')

In [ ]:
lc1 = hist_df['histogram'].iloc[1000]

plt.imshow(lc1 / lc1.max())

In [ ]:
plt.imshow(lc1[:, :, 0])

In [ ]:
obj_name = hist_df['name'].iloc[45]
obj_df = all_lightcurves[(all_lightcurves['name'] == obj_name) & (all_lightcurves['band'] == 'r')]
obj_df = obj_df.sort_values(by='mjd')

mags, times = obj_df['mag'].to_numpy(), obj_df['mjd'].to_numpy()
dm, dt = return_dms_dts(mags, times)


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(13, 5))
axs[0].scatter(times, mags)
axs[0].set_title('Object Magnitude vs. Times')
axs[0].set_xlabel('Times')
axs[0].set_ylabel('Mag')

axs[1].plot(dt, dm, 'ko')
axs[1].set_title('Object $dm$ vs. $dt$')
axs[1].set_xlabel('$dt$')
axs[1].set_ylabel('$dm$')

In [ ]:
plt.hexbin(dt, dm, cmap='jet')

In [ ]:
hist, _, _ = np.histogram2d(dt, dm, bins=[23, 24])
plt.imshow(hist.T / hist.sum(), origin='lower', interpolation='bilinear')

In [ ]:
plt.imshow(hist_df['histogram'].iloc[54])

In [ ]:
plot_dm_dt_rgb(hist_df['histogram'].iloc[54], dm_bins, dt_bins)

In [ ]:
plot_dm_dt(hist_df['histogram'].iloc[54][:, :, 0], band='r', dm_bins=dm_bins, dt_bins=dt_bins, title='Random Light Curve')

In [ ]:
tot_Qs = hist_df[hist_df['type'] == 'Q']['histogram'].sum()
tot_QRs = hist_df[hist_df['type'] == 'QR']['histogram'].sum()
tot_QXs = hist_df[hist_df['type'] == 'QX']['histogram'].sum()

to_plot = [tot_Qs, tot_QRs, tot_QXs]
titles = ['All Qs', 'All QRs', 'All QXs']
dm_nticks, dt_nticks = 10, 10

dm_indices = linspace(0, len(dm_bins) - 1, dm_nticks, dtype=int)
dt_indices = linspace(0, len(dt_bins) - 1, dt_nticks, dtype=int)

dm_ticks_labels = []
for index in dm_indices:
    formatted_string = f"{dm_bins[index]:.2f}"
    dm_ticks_labels.append(formatted_string)

dt_ticks_labels = []
for index in dt_indices:
    formatted_string = f"{dt_bins[index]:.1e}"
    base, exponent = formatted_string.split('e')
    dt_ticks_labels.append(f"${base} \\times 10^{{{int(exponent)}}}$")


fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

for hist, ax, title in zip(to_plot, axs, titles):
    ax.imshow(hist / hist.max(), extent=[0, len(dt_bins)-1, 0, len(dm_bins) - 1])
    ax.set_yticks(ticks=dm_indices, labels=dm_ticks_labels)
    ax.set_xticks(ticks=dt_indices, labels=dt_ticks_labels, size=7)
    ax.set_title(title)

fig.supxlabel(f'{dt_bins.min():.2f} $< dt <$ {dt_bins.max():.2f} (days) {len(dt_bins) - 1} bins', size=12)
fig.supylabel(f'{dm_bins.min():.2f} $< dm <$ {dm_bins.max():.2f} (magnitude) {len(dm_bins) - 1} bins', size=12, x=0.0001)

plt.tight_layout()

In [ ]:
tot_Qs = hist_df[hist_df['type'] == 'Q']['histogram'].sum()
tot_QRs = hist_df[hist_df['type'] == 'QR']['histogram'].sum()
tot_QXs = hist_df[hist_df['type'] == 'QX']['histogram'].sum()

dm_hsit_Q = np.array([tot_Qs[i, :, :].sum() for i in range(tot_Qs.shape[0])])
dm_hsit_Q = np.array([tot_Qs[i, :, :].sum() for i in range(tot_Qs.shape[0])])
dm_hsit_Q = np.array([tot_Qs[i, :, :].sum() for i in range(tot_Qs.shape[0])])

to_plot = [tot_Qs, tot_QRs, tot_QXs]

titles = ['All Qs', 'All QRs', 'All QXs']
dm_nticks, dt_nticks = 10, 10

to_plot2 = [dm_hsit_Q, dm_hsit_Q, dm_hsit_Q]
titles = ['All Qs', 'All QRs', 'All QXs']

dm_indices = linspace(0, len(dm_bins) - 1, dm_nticks, dtype=int)
dt_indices = linspace(0, len(dt_bins) - 1, dt_nticks, dtype=int)

dm_ticks_labels = []
for index in dm_indices:
    formatted_string = f"{dm_bins[index]:.2f}"
    dm_ticks_labels.append(formatted_string)

dt_ticks_labels = []
for index in dt_indices:
    formatted_string = f"{dt_bins[index]:.1e}"
    base, exponent = formatted_string.split('e')
    dt_ticks_labels.append(f"${base} \\times 10^{{{int(exponent)}}}$")


fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(19, 11))

for hist, ax, title in zip(to_plot, axs[0, :], titles):
    ax.imshow(hist / hist.max(), extent=[0, len(dt_bins)-1, 0, len(dm_bins) - 1])
    ax.set_yticks(ticks=dm_indices, labels=dm_ticks_labels)
    ax.set_xticks(ticks=dt_indices, labels=dt_ticks_labels, size=7)
    ax.set_title(title)
    ax.set_xlabel(f'{dt_bins.min():.2f} $< dt <$ {dt_bins.max():.2f} (days) {len(dt_bins) - 1} bins', size=12)
axs[0, 0].set_ylabel(f'{dm_bins.min():.2f} $< dm <$ {dm_bins.max():.2f} (magnitude) {len(dm_bins) - 1} bins', size=12, x=0.00001)

for distribution, ax, title in zip(to_plot2, axs[1, :], titles):
    ax.plot((dm_bins[:-1] + dm_bins[1:]) / 2, distribution)
    ax.set_xlabel(f'{dm_bins.min():.2f} $< dm <$ {dm_bins.max():.2f} (magnitude) {len(dm_bins) - 1} bins', size=12)
axs[1, 0].set_ylabel('Distribution of $dm$ summed across all bands and all $dt$ bins')


plt.tight_layout()

In [ ]:
tot_Qs = hist_df[hist_df['type'] == 'Q']['histogram'].sum()
tot_QRs = hist_df[hist_df['type'] == 'QR']['histogram'].sum()
tot_QXs = hist_df[hist_df['type'] == 'QX']['histogram'].sum()

tot_Qs = tot_Qs / tot_Qs.sum()
tot_QRs = tot_QRs / tot_QRs.sum()
tot_QXs = tot_QXs / tot_QXs.sum()

diff_Q_QR = tot_Qs - tot_QRs
diff_Q_QX = tot_Qs - tot_QXs
diff_QR_QX = tot_QRs - tot_QXs

to_plot = [diff_Q_QR, diff_Q_QX, diff_QR_QX]
titles = ['|Q - QR|', '|Q - QX|', '|QR - QX|']
dm_nticks, dt_nticks = 10, 10

dm_indices = linspace(0, len(dm_bins) - 1, dm_nticks, dtype=int)
dt_indices = linspace(0, len(dt_bins) - 1, dt_nticks, dtype=int)

dm_ticks_labels = []
for index in dm_indices:
    formatted_string = f"{dm_bins[index]:.2f}"
    dm_ticks_labels.append(formatted_string)

dt_ticks_labels = []
for index in dt_indices:
    formatted_string = f"{dt_bins[index]:.1e}"
    base, exponent = formatted_string.split('e')
    dt_ticks_labels.append(f"${base} \\times 10^{{{int(exponent)}}}$")


fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

for hist, ax, title in zip(to_plot, axs, titles):
    hist = (hist - hist[:, :, :-1].min()) / (hist[:, :, :-1].max() - hist[:, :, :-1].min())
    hist[:, :, 2] = np.zeros_like(hist[:, :, 2])
    ax.imshow(hist, extent=[0, len(dt_bins)-1, 0, len(dm_bins) - 1])
    ax.set_yticks(ticks=dm_indices, labels=dm_ticks_labels)
    ax.set_xticks(ticks=dt_indices, labels=dt_ticks_labels, size=7)
    ax.set_title(title)

fig.supxlabel(f'{dt_bins.min():.2f} $< dt <$ {dt_bins.max():.2f} (days) {len(dt_bins) - 1} bins', size=12)
fig.supylabel(f'{dm_bins.min():.2f} $< dm <$ {dm_bins.max():.2f} (magnitude) {len(dm_bins) - 1} bins', size=12)

plt.tight_layout()

In [ ]:
dm_bins

In [ ]:
def plot_n_objects(all_lightcurves, object_type, n, band, save_path='subplots.png'):
    df = all_lightcurves[(all_lightcurves['type'] == object_type) & (all_lightcurves['band'] == band)].copy()
    df = df.sort_values(by=['name', 'mjd'])
    names = np.random.choice(df['name'].unique(), n, replace=False)

    total_m, total_t = [], []
    total_dm, total_dt = [], []

    fig, axes = plt.subplots(n+1, 5, figsize=(19, 4 * n))

    # If n is 1, axes will be a 1D array; make it 2D for consistency
    if n == 1:
        axes = axes[np.newaxis, :]

    # Generate plots for each row
    for i, name in enumerate(names):
        mags, times = df[df['name'] == name]['mag'].to_numpy(), df[df['name'] == name]['mjd'].to_numpy()
        dm, dt = return_dms_dts(mags, times)
        total_dm = np.r_[total_dm, dm]
        total_dt = np.r_[total_dt, dt]
        total_m = np.r_[total_m, mags]
        total_t = np.r_[total_t, times]

        axes[i, 0].scatter(times, mags, s=1)
        axes[i, 0].set_ylabel(f'{object_type} Object: {name}')

        axes[i, 1].scatter(dt, dm, s=1)
        axes[i, 1].set_xlim(dt_bins[0], dt_bins[-1])
        axes[i, 1].set_ylim(dm_bins[0], dm_bins[-1])

        hist, _, _ = np.histogram2d(dt, dm, bins=[dt_bins, dm_bins])
        axes[i, 2].imshow(hist.T, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], aspect='auto', cmap='viridis')

        axes[i, 3].hexbin(dt, dm, mincnt=1, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], cmap='viridis')

        axes[i, 4].hexbin(dt, dm, bins='log', mincnt=1, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], cmap='viridis')


    axes[n, 0].scatter(total_t, total_m, s=1)
    axes[n, 0].set_ylabel('Total')

    axes[n, 1].scatter(total_dt, total_dm, s=1)
    axes[n, 1].set_xlim(dt_bins[0], dt_bins[-1])
    axes[n, 1].set_ylim(dm_bins[0], dm_bins[-1])

    hist, _, _ = np.histogram2d(total_dt, total_dm, bins=[dt_bins, dm_bins])
    axes[n, 2].imshow(hist.T, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], aspect='auto', cmap='viridis')

    axes[n, 3].hexbin(total_dt, total_dm, mincnt=1, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], cmap='viridis')

    axes[n, 4].hexbin(total_dt, total_dm, bins='log', mincnt=1, extent=[dt_bins[0], dt_bins[-1], dm_bins[0], dm_bins[-1]], cmap='viridis')


    # Adjust layout
    plt.tight_layout()

    # Save the resulting plot grid to disk
    # lt.savefig(save_path)
    # print(f"Plot saved to {save_path}")

    # Display the plot
    plt.show()

In [ ]:
plot_n_objects(all_lightcurves, 'Q', 3, 'g')